# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 14 2022 1:55PM,248671,19,ADV80006566,"AdvaGen Pharma, Ltd",Released
1,Oct 14 2022 1:55PM,248671,19,ADV80006567,"AdvaGen Pharma, Ltd",Released
2,Oct 14 2022 1:31PM,248668,10,0086076747,ISDIN Corporation,Released
3,Oct 14 2022 1:31PM,248668,10,0086076756,ISDIN Corporation,Released
4,Oct 14 2022 1:31PM,248668,10,0086076757,ISDIN Corporation,Released
5,Oct 14 2022 1:31PM,248668,10,0086076759,ISDIN Corporation,Released
6,Oct 14 2022 1:31PM,248668,10,0086076758,ISDIN Corporation,Released
7,Oct 14 2022 1:31PM,248668,10,0086076761,ISDIN Corporation,Released
8,Oct 14 2022 1:31PM,248668,10,0086076762,ISDIN Corporation,Released
9,Oct 14 2022 1:31PM,248668,10,0086076768,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,248662,Released,2
22,248665,Released,1
23,248667,Released,7
24,248668,Released,21
25,248671,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248662,NaN,1.0,2.0
248665,NaN,NaN,1.0
248667,NaN,NaN,7.0
248668,NaN,NaN,21.0
248671,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248103,0.0,1.0,0.0
248109,0.0,1.0,0.0
248267,0.0,18.0,3.0
248610,0.0,0.0,23.0
248615,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248103,0,1,0
248109,0,1,0
248267,0,18,3
248610,0,0,23
248615,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248103,0,1,0
1,248109,0,1,0
2,248267,0,18,3
3,248610,0,0,23
4,248615,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248103,,1,
1,248109,,1,
2,248267,,18,3
3,248610,,,23
4,248615,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 14 2022 1:55PM,248671,19,"AdvaGen Pharma, Ltd"
2,Oct 14 2022 1:31PM,248668,10,ISDIN Corporation
23,Oct 14 2022 1:27PM,248667,10,ISDIN Corporation
30,Oct 14 2022 1:00PM,248665,10,"Senseonics, Incorporated"
31,Oct 14 2022 12:42PM,248662,15,"Virtus Pharmaceuticals, LLC"
34,Oct 14 2022 12:36PM,248659,15,"Person & Covey, Inc."
51,Oct 14 2022 12:28PM,248657,15,"Person & Covey, Inc."
67,Oct 14 2022 12:12PM,248267,15,"Person & Covey, Inc."
88,Oct 14 2022 12:07PM,248648,10,ISDIN Corporation
93,Oct 14 2022 11:59AM,248647,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 14 2022 1:55PM,248671,19,"AdvaGen Pharma, Ltd",,,2
1,Oct 14 2022 1:31PM,248668,10,ISDIN Corporation,,,21
2,Oct 14 2022 1:27PM,248667,10,ISDIN Corporation,,,7
3,Oct 14 2022 1:00PM,248665,10,"Senseonics, Incorporated",,,1
4,Oct 14 2022 12:42PM,248662,15,"Virtus Pharmaceuticals, LLC",,1,2
5,Oct 14 2022 12:36PM,248659,15,"Person & Covey, Inc.",,15,2
6,Oct 14 2022 12:28PM,248657,15,"Person & Covey, Inc.",,14,2
7,Oct 14 2022 12:12PM,248267,15,"Person & Covey, Inc.",,18,3
8,Oct 14 2022 12:07PM,248648,10,ISDIN Corporation,,1,4
9,Oct 14 2022 11:59AM,248647,19,ACG North America LLC,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 1:55PM,248671,19,"AdvaGen Pharma, Ltd",2,,
1,Oct 14 2022 1:31PM,248668,10,ISDIN Corporation,21,,
2,Oct 14 2022 1:27PM,248667,10,ISDIN Corporation,7,,
3,Oct 14 2022 1:00PM,248665,10,"Senseonics, Incorporated",1,,
4,Oct 14 2022 12:42PM,248662,15,"Virtus Pharmaceuticals, LLC",2,1,
5,Oct 14 2022 12:36PM,248659,15,"Person & Covey, Inc.",2,15,
6,Oct 14 2022 12:28PM,248657,15,"Person & Covey, Inc.",2,14,
7,Oct 14 2022 12:12PM,248267,15,"Person & Covey, Inc.",3,18,
8,Oct 14 2022 12:07PM,248648,10,ISDIN Corporation,4,1,
9,Oct 14 2022 11:59AM,248647,19,ACG North America LLC,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 1:55PM,248671,19,"AdvaGen Pharma, Ltd",2,,
1,Oct 14 2022 1:31PM,248668,10,ISDIN Corporation,21,,
2,Oct 14 2022 1:27PM,248667,10,ISDIN Corporation,7,,
3,Oct 14 2022 1:00PM,248665,10,"Senseonics, Incorporated",1,,
4,Oct 14 2022 12:42PM,248662,15,"Virtus Pharmaceuticals, LLC",2,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 1:55PM,248671,19,"AdvaGen Pharma, Ltd",2.0,NaN,NaN
1,Oct 14 2022 1:31PM,248668,10,ISDIN Corporation,21.0,NaN,NaN
2,Oct 14 2022 1:27PM,248667,10,ISDIN Corporation,7.0,NaN,NaN
3,Oct 14 2022 1:00PM,248665,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Oct 14 2022 12:42PM,248662,15,"Virtus Pharmaceuticals, LLC",2.0,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 1:55PM,248671,19,"AdvaGen Pharma, Ltd",2.0,0.0,0.0
1,Oct 14 2022 1:31PM,248668,10,ISDIN Corporation,21.0,0.0,0.0
2,Oct 14 2022 1:27PM,248667,10,ISDIN Corporation,7.0,0.0,0.0
3,Oct 14 2022 1:00PM,248665,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Oct 14 2022 12:42PM,248662,15,"Virtus Pharmaceuticals, LLC",2.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2237754,68.0,1.0,2.0
15,994245,9.0,48.0,0.0
16,497269,1.0,1.0,2.0
19,497318,4.0,0.0,0.0
21,496212,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2237754,68.0,1.0,2.0
1,15,994245,9.0,48.0,0.0
2,16,497269,1.0,1.0,2.0
3,19,497318,4.0,0.0,0.0
4,21,496212,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,68.0,1.0,2.0
1,15,9.0,48.0,0.0
2,16,1.0,1.0,2.0
3,19,4.0,0.0,0.0
4,21,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,68.0
1,15,Released,9.0
2,16,Released,1.0
3,19,Released,4.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,2.0,0.0,2.0,0.0,0.0
Executing,1.0,48.0,1.0,0.0,2.0
Released,68.0,9.0,1.0,4.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,2.0,0.0,2.0,0.0,0.0
1,Executing,1.0,48.0,1.0,0.0,2.0
2,Released,68.0,9.0,1.0,4.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,2.0,0.0,2.0,0.0,0.0
1,Executing,1.0,48.0,1.0,0.0,2.0
2,Released,68.0,9.0,1.0,4.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()